# Preprocessing

In [1]:
import pandas as pd #Gestionar dataframes
import numpy as np #Numeric python
import matplotlib.pyplot as plt #Graficos
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from datetime import datetime, date

In [2]:
tr_performance = pd.read_csv("train_performance.csv", sep=',', decimal=';')
tr_datos_demog = pd.read_csv("train_datos_demograficos.csv", sep=',', decimal=';')
tr_prev_loan = pd.read_csv("train_previous_loan.csv", sep=',', decimal=';')

Merging dataset performance and demographic data with the primary key, customerid

In [9]:
df = tr_datos_demog.merge(tr_performance, how='inner', on='customerid')
#print(df)

Make our target variable into binary (good_bad_flag into binary)and we copy all the info into another dataframe as original must be left untouched

In [10]:
config_df = df.copy()
config_df["good_bad_flag"] = config_df["good_bad_flag"].apply(lambda x: 0 if x=='Bad' else 1) #Changing good to 1, bad to 0 
config_df["good_bad_flag"].value_counts()

1    2563
0     714
Name: good_bad_flag, dtype: int64

Due to EDA, we know the variable referredby variable has an 87% of missing variables. We will convert referred_by to binary, referred (1) or not referred (0)

In [11]:
config_df["referredby"].value_counts()
config_df['referredby'] = config_df['referredby'].fillna(0)
config_df['referred'] = config_df['referredby'].apply(lambda x: 0 if x==0 else 1) #Changing strings to 1 (referred), everything to 0 (not referred)
config_df.drop('referredby', axis=1, inplace=True)

Turning birthdate into age so we can use it as a variable

In [13]:
config_df['age'] = pd.to_datetime(config_df.birthdate)

def from_dob_to_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

config_df['realage'] = config_df['age'].apply(lambda x: from_dob_to_age(x))
config_df.drop(['age', 'birthdate'], axis=1, inplace=True)

We need to change variables: loanamount and totaldue to floats instead of objects

In [15]:
config_df.dtypes
config_df['loanamount'].isna().sum()
config_df['totaldue'].isna().sum()

config_df['loanamount'] = pd.to_numeric(config_df['loanamount'],errors = 'coerce')
config_df['totaldue'] = pd.to_numeric(config_df['totaldue'],errors = 'coerce')


Now we are going to try and find out with latitude and longitude the places where our clients are. Refer to our github Geocoder file



```
# Code used but ultimately dropped the columns & variables

import geocoder
config_df['lat'].dropna()
config_df['long'].dropna()
config_df['latitude_gps'] = pd.to_numeric(config_df['latitude_gps'])
config_df['long'] = pd.to_numeric(config_df['long'], downcast="float")
def geo_rev(x):
    g = geocoder.osm([x.lat, x.long], method='reverse').json
    if g:
        return g.get('country')
    else:
        return 'no country'
        
config_df[['lat', 'long']].apply(geo_rev, axis=1)
```


In [18]:
config_df.drop(['latitude_gps', 'longitude_gps'], axis=1, inplace=True) 

Due to EDA, we know bank_branch_clients variable has a 99% of missing variables. We will convert bank_branch_clients to binary, has a bank branch or not






In [20]:
config_df['bank_branch_clients'].unique()
config_df['bank_branch_clients'].describe()
config_df['bank_branch_clients'].isna().sum()
config_df['bank_branch_clients'] = config_df['bank_branch_clients'].fillna('Unknown')

config_df['bank_branch_clients'] = config_df['bank_branch_clients'].apply(lambda x: 0 if x== 'Unknown' else 1) 

In [21]:
config_df.head()

,customerid,bank_account_type,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,good_bad_flag,referred,realage
0,8a858e135cb22031015cbafc76964ebd,Savings,GT Bank,1,NaN,NaN,301964962,2,2017-07-05 14:29:48.000000,2017-07-05 13:29:42.000000,10000.0,13000.0,30,1,1,48
1,8a858e275c7ea5ec015c82482d7c3996,Savings,Sterling Bank,1,Permanent,NaN,301972172,2,2017-07-10 21:21:46.000000,2017-07-10 20:21:40.000000,10000.0,13000.0,30,0,0,36
2,8a858e5b5bd99460015bdc95cd485634,Savings,Fidelity Bank,1,NaN,NaN,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,1,0,35
3,8a858efd5ca70688015cabd1f1e94b55,Savings,GT Bank,1,Permanent,NaN,301997763,2,2017-07-26 21:03:17.000000,2017-07-26 20:03:09.000000,10000.0,11500.0,15,1,0,30
4,8a858ea05a859123015a8892914d15b7,Savings,Access Bank,1,Permanent,NaN,301992494,6,2017-07-23 21:44:43.000000,2017-07-23 20:44:36.000000,20000.0,24500.0,30,1,0,31


Making a transformation, and finding out the interest/rate of the loan. Counting how the rate of the loan is now


In [23]:
config_df['TipoInteresAhora'] = ((config_df['totaldue'] - config_df['loanamount']) / (config_df['loanamount'] * config_df['termdays']))*100

Making a transformation between creation date and approved date. We leave the code here, that attempts where made to extract insights but, there was no insight as the difference was too small and there was no variation



```
# Code used but ultimately dropped the columns & variables

Time difference between approveddate - creationdate and
 test dataframe; the columns must be in a datetime format;# test dataframe; the columns must be in a datetime format;
for col in config_df.columns:
    if config_df[col].dtype == 'object':
        try:
            config_df[col] = pd.to_datetime(config_df[col])
        except ValueError:
            pass

config_df.dtypes
config_df['difference_time'] = (config_df.approveddate - config_df.creationdate)
#Create a column with timedelta as total hours, as a float type
config_df['tot_hour_diff'] = (config_df.approveddate - config_df.creationdate) / pd.Timedelta(hours=1)
#(This last column is in hours)
config_df.drop(['difference_time', 'approveddate', 'creationdate'], axis=1, inplace=True) 
config_df['tot_hour_diff'].describe()
```



In [24]:
config_df.drop(['creationdate', 'approveddate'], axis=1, inplace=True) 

Now, we will use onehotencoding to make categorical data into numerical 
Firstly, we will be reprocessing to be able to do it and nan valueswill be transformed into another category 'Unknown'

In [28]:
#Bank account type variable
print(config_df['bank_account_type'].unique())
config_df['bank_account_type'].isna().sum()

['Savings' 'Other' 'Current']


0

In [29]:
#Bank name clients variable
print(config_df['bank_name_clients'].unique())
config_df['bank_name_clients'].isna().sum()

['GT Bank' 'Sterling Bank' 'Fidelity Bank' 'Access Bank' 'EcoBank' 'FCMB'
 'Skye Bank' 'UBA' 'Diamond Bank' 'Zenith Bank' 'First Bank' 'Union Bank'
 'Stanbic IBTC' 'Standard Chartered' 'Heritage Bank' 'Keystone Bank'
 'Unity Bank' 'Wema Bank']


0

In [32]:
#Employment status clients variable
#print(config_df['employment_status_clients'].unique())
config_df['employment_status_clients'].describe()
config_df['employment_status_clients'].isna().sum()
config_df['employment_status_clients'] = config_df['employment_status_clients'].fillna('Unknown')
print(config_df['employment_status_clients'].unique())


['Unknown' 'Permanent' 'Student' 'Self-Employed' 'Unemployed' 'Retired'
 'Contract']


In [34]:
#Level of education clients variable
#print(config_df['level_of_education_clients'].unique())
config_df['level_of_education_clients'].describe()
config_df['level_of_education_clients'].isna().sum()
config_df['level_of_education_clients'] = config_df['level_of_education_clients'].fillna('Unknown')
print(config_df['level_of_education_clients'].unique())


['Unknown' 'Secondary' 'Graduate' 'Post-Graduate' 'Primary']


Putting all categorical values into a separate dataframe to make onehotencoding




In [35]:
encoded_df = config_df[['bank_account_type', 'bank_name_clients', 'employment_status_clients', 'level_of_education_clients']].copy()

In [36]:
#OneHotEncoding
ohe = OneHotEncoder(sparse = False)
ohe_fit = ohe.fit(encoded_df)
X_ohe = pd.DataFrame(ohe.fit_transform(encoded_df))
X_ohe.columns = pd.DataFrame(ohe_fit.get_feature_names())
ohe.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['x0_Current', 'x0_Other', 'x0_Savings', 'x1_Access Bank',
       'x1_Diamond Bank', 'x1_EcoBank', 'x1_FCMB', 'x1_Fidelity Bank',
       'x1_First Bank', 'x1_GT Bank', 'x1_Heritage Bank',
       'x1_Keystone Bank', 'x1_Skye Bank', 'x1_Stanbic IBTC',
       'x1_Standard Chartered', 'x1_Sterling Bank', 'x1_UBA',
       'x1_Union Bank', 'x1_Unity Bank', 'x1_Wema Bank', 'x1_Zenith Bank',
       'x2_Contract', 'x2_Permanent', 'x2_Retired', 'x2_Self-Employed',
       'x2_Student', 'x2_Unemployed', 'x2_Unknown', 'x3_Graduate',
       'x3_Post-Graduate', 'x3_Primary', 'x3_Secondary', 'x3_Unknown'],
      dtype=object)

Renaming after onehot encoding

In [40]:
#Renaming bank account type
bank_account_df = X_ohe.copy()
bank_account_df.drop(bank_account_df.iloc[:, 3:], inplace = True, axis = 1)

#To take out the tuple of thee name column 
def rename(col):
    if isinstance(col, tuple):
        col = '_'.join(str(c) for c in col)
    return col


bank_account_df.columns = map(rename, bank_account_df.columns)
bank_account_df.columns = bank_account_df.columns.str.replace('x0', 'bank_account')
#print(bank_account_df)

      bank_account_Current  bank_account_Other  bank_account_Savings
0                      0.0                 0.0                   1.0
1                      0.0                 0.0                   1.0
2                      0.0                 0.0                   1.0
3                      0.0                 0.0                   1.0
4                      0.0                 0.0                   1.0
...                    ...                 ...                   ...
3272                   0.0                 0.0                   1.0
3273                   0.0                 0.0                   1.0
3274                   0.0                 1.0                   0.0
3275                   0.0                 1.0                   0.0
3276                   0.0                 0.0                   1.0

[3277 rows x 3 columns]


In [41]:
#Renaming level of education clients
level_of_edu_df = X_ohe.copy()
level_of_edu_df.drop(level_of_edu_df.iloc[:, :-5], inplace = True, axis = 1)
level_of_edu_df.columns = map(rename, level_of_edu_df.columns)
level_of_edu_df.columns = level_of_edu_df.columns.str.replace('x3', 'level_of_education_clients')
print(level_of_edu_df)

      level_of_education_clients_Graduate  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   
...                                   ...   
3272                                  0.0   
3273                                  0.0   
3274                                  0.0   
3275                                  1.0   
3276                                  0.0   

      level_of_education_clients_Post-Graduate  \
0                                          0.0   
1                                          0.0   
2                                          0.0   
3                                          0.0   
4                                          0.0   
...                                        ...   
3272                                       0.0   
3273                                       0.0   
3274     

In [42]:
#Renaming employment status clients
employment_status_client_df = X_ohe.copy()
employment_status_client_df = employment_status_client_df.iloc[:, 21:28]
employment_status_client_df.columns = map(rename, employment_status_client_df.columns)
employment_status_client_df.columns = employment_status_client_df.columns.str.replace('x2', 'employment_status_client')
print(employment_status_client_df)

      employment_status_client_Contract  employment_status_client_Permanent  \
0                                   0.0                                 0.0   
1                                   0.0                                 1.0   
2                                   0.0                                 0.0   
3                                   0.0                                 1.0   
4                                   0.0                                 1.0   
...                                 ...                                 ...   
3272                                0.0                                 1.0   
3273                                0.0                                 0.0   
3274                                0.0                                 0.0   
3275                                0.0                                 1.0   
3276                                0.0                                 0.0   

      employment_status_client_Retired  \
0        

In [43]:
#Renaming bank name clients
bank_name_client_df = X_ohe.copy()
bank_name_client_df = bank_name_client_df.iloc[:, 3:21]
bank_name_client_df.columns = map(rename, bank_name_client_df.columns)
bank_name_client_df.columns = bank_name_client_df.columns.str.replace('x1', 'bank_name_client')
print(bank_name_client_df)

      bank_name_client_Access Bank  bank_name_client_Diamond Bank  \
0                              0.0                            0.0   
1                              0.0                            0.0   
2                              0.0                            0.0   
3                              0.0                            0.0   
4                              1.0                            0.0   
...                            ...                            ...   
3272                           0.0                            0.0   
3273                           0.0                            0.0   
3274                           0.0                            0.0   
3275                           0.0                            0.0   
3276                           0.0                            0.0   

      bank_name_client_EcoBank  bank_name_client_FCMB  \
0                          0.0                    0.0   
1                          0.0                    0.0   


Join all the one hot encoded dataframes




In [44]:
result = pd.concat([bank_account_df, level_of_edu_df , bank_name_client_df, employment_status_client_df ], axis=1, join='inner')

In [45]:
#Delete auxiliaries dataframes
del(encoded_df, bank_name_client_df , level_of_edu_df , bank_account_df, employment_status_client_df )
#Check X_ohe dataframe is the same as result dataframe and delete
del(X_ohe, ohe, ohe_fit)

Merge dataframes 



In [46]:
config_df.drop(['bank_account_type', 'bank_name_clients', 'employment_status_clients', 'level_of_education_clients' ], axis=1, inplace=True) 

In [47]:
#Merging config_df with one hot encoded dataframe
merged_df = pd.concat([config_df, result ], axis=1, join='inner')

In [48]:
#Delete auxiliaries dataframes
del(config_df, result)

Now, lets work on the tr_prev_loan and make transformation to extract insights and not data.

In [51]:
for col in tr_prev_loan.columns:
    if tr_prev_loan[col].dtype == 'object':
        try:
            tr_prev_loan[col] = pd.to_datetime(tr_prev_loan[col])
        except ValueError:
            pass

In [52]:
#We want to know how much in hours did they take to take up/use all the loan
#closeddate - approveddate

tr_prev_loan['how_long_loan_time'] = (tr_prev_loan.closeddate - tr_prev_loan.approveddate) # create a column with timedelta as total hours, as a float type
tr_prev_loan['how_long_consumed_loan_time'] = (tr_prev_loan.closeddate - tr_prev_loan.approveddate) / pd.Timedelta(hours=1) #(This last column is in hours)
tr_prev_loan['hours_termdays'] = tr_prev_loan['termdays'].apply(lambda x: x*24)
tr_prev_loan['exceeds_loan_term_days'] = np.where((tr_prev_loan['hours_termdays'] >= tr_prev_loan['how_long_consumed_loan_time']), 0, 1) #Exceeds time 1, not exceed time 0
tr_prev_loan.drop(['hours_termdays', 'how_long_consumed_loan_time', 'how_long_loan_time', 'closeddate', 'approveddate' ], axis=1, inplace=True) 


In [54]:
#Now, we want to know how much late did they pay the first time hence,
# firstduedate - firstpaiddate

tr_prev_loan['difference_time'] = (tr_prev_loan.firstduedate - tr_prev_loan.firstrepaiddate)
# create a column with timedelta as total hours, as a float type
tr_prev_loan['diff_duedate_paiddate'] = (tr_prev_loan.firstduedate - tr_prev_loan.firstrepaiddate) / pd.Timedelta(hours=1)
#(This last column is in hours)
tr_prev_loan['is_late_for_firstpay'] = [1 if x < 0 else 0 for x in tr_prev_loan.diff_duedate_paiddate]
# 1 means he was late, 0 he was not
tr_prev_loan.drop(['firstduedate', 'firstrepaiddate', 'difference_time', 'diff_duedate_paiddate' ], axis=1, inplace=True) 





```
# Code used but ultimately dropped the columns & variables

#Which months where the loans asked for?

tr_prev_loan['month_approved_loan'] = tr_prev_loan['approveddate'].dt.month_name()
tr_prev_loan.drop(['creationdate'], axis=1, inplace=True) 
```



Finally, we want to know how much interest/rate was given to the clients PREVIOUSLY for the amount given.




In [56]:
tr_prev_loan['loanamount'] = pd.to_numeric(tr_prev_loan['loanamount'],errors = 'coerce')
tr_prev_loan['totaldue'] = pd.to_numeric(tr_prev_loan['totaldue'],errors = 'coerce')
#Calculating the interest given to the client
tr_prev_loan['TipoInteres'] = ((tr_prev_loan['totaldue'] - tr_prev_loan['loanamount']) / (tr_prev_loan['loanamount'] * tr_prev_loan['termdays']))*100


In [57]:
#Dropping the variable systemloanid
merged_df.drop(['systemloanid'], axis=1, inplace=True) 

In [62]:
# We create a new dataframe to merge into merged_df
loan = tr_prev_loan[['customerid', 'is_late_for_firstpay', 'exceeds_loan_term_days', 'TipoInteres' ]].copy()

In [63]:
finalmerge = merged_df.merge(loan, on="customerid", how="left")

In [64]:
finalmerge.to_csv('finalmerge.csv')